In [ ]:
%%javascript
document.title="local"

In [ ]:
from skimage import data, transform, io
from skimage.color import rgb2gray
import numpy as np
import pandas as pd
import napari
from napari.utils.transforms import Affine, CompositeAffine
import matplotlib.pyplot as plt
from MILWRM.MxIF import *

In [ ]:
sample_key = pd.read_csv("/Users/cody/Dropbox (VU Basic Sciences)/fuzzy-lasagna/resources/visium_sample_key.csv", index_col=0)

sample_key.ws_adata = sample_key.ws_adata.fillna("")

---
# Read in two images for registration

In [ ]:
slide_id = "SG00002"
visium_id = "8899_4"

In [ ]:
sample_key.loc[sample_key.patient_name=="SG00002", ["sample_key","ws_he","ws_flip"]]

In [ ]:
%time a = img.from_npz("/Users/cody/Downloads/{}_downsample8.npz".format(slide_id))

In [ ]:
rgb = a.show(channels=["PANCK","BCATENIN","SMA"], RGB=True)

In [ ]:
ST = io.imread("/Users/cody/Dropbox (VU Basic Sciences)/fuzzy-lasagna/data/visium/datasets/8899_AS/images_alignment/V11U23-321_D1 copy.tif")

In [ ]:
plt.imshow(ST)#[::-1,::-1,:])

### Set up napari viewer

In [ ]:
A = a["PANCK","BCATENIN","SMA"].copy()

In [ ]:
# flip the MxIF image horizontally (if needed)
A = A[::-1,::-1,:]

In [ ]:
# flip the MxIF image horizontally (if needed)
ST = ST[::-1,::-1,:]

In [ ]:
def on_transform_changed_drag(event):
    """record affine transform as ST is manipulated"""
    viewer.layers.selection.active.affine = event.value
    
viewer = napari.view_image(A)  # whole-slide image to register to ST space
viewer.add_image(ST, opacity=0.5)  # ST on top with transparency
# set up interaction box for moving/scaling ST image
viewer.layers.selection.active.interactive = False
viewer.overlays.interaction_box.points = viewer.layers.selection.active.extent.world
viewer.overlays.interaction_box.show = True
viewer.overlays.interaction_box.show_vertices = True
viewer.overlays.interaction_box.show_handle = True
viewer.overlays.interaction_box.allow_new_selection = False
viewer.overlays.interaction_box.events.transform_drag.connect(on_transform_changed_drag)
# open viewer
napari.run()

### Extract applied affine transformation

In [ ]:
#Extract inverse of affine
cut = Affine(
    rotate=viewer.layers.selection.active.affine.inverse.rotate,
    translate=viewer.layers.selection.active.affine.translate[::-1],
)
scale = (
    viewer.layers.selection.active.affine.scale[0],
    viewer.layers.selection.active.affine.scale[1],
    1,  # for 3D image
)
shape = np.array(
    np.array(ST.shape, dtype = np.float64) * scale,
    dtype = int,
)  # keep original aspect ratio

In [ ]:
# save affine matrix and final shape to files so we don't have to re-do the manual registration in the future
np.savetxt(
    X=cut.affine_matrix,
    fname="/Users/cody/Downloads/{}_downsample8_{}_affine2.csv".format(slide_id,visium_id),
    delimiter=",",
)
np.savetxt(
    X=shape,
    fname="/Users/cody/Downloads/{}_downsample8_{}_shape2.csv".format(slide_id,visium_id),
    delimiter=",",
)

#### save affine matrix to variable

In [ ]:
affine = cut.affine_matrix

### Pull image out of background and export

In [ ]:
affine

In [ ]:
shape

In [ ]:
image = transform.warp(A, transform.AffineTransform(affine), output_shape=abs(shape))

In [ ]:
plt.imshow(image)